### Data preparation
#### load and prepare the data set

In [8]:
import pandas as pd 

data = pd.read_csv('Restaurant_reviews.csv')

data = data[['Review', 'Rating']]. dropna().reset_index(drop=True)

#create new column for labeling each aspect
data['Food Quality Sentiment'] = ''
data['Service Sentiment'] = ''
data['Ambiance Sentiment'] = ''

print(data.head())

                                              Review Rating  \
0  The ambience was good, food was quite good . h...      5   
1  Ambience is too good for a pleasant evening. S...      5   
2  A must try.. great food great ambience. Thnx f...      5   
3  Soumen das and Arun was a great guy. Only beca...      5   
4  Food is good.we ordered Kodi drumsticks and ba...      5   

  Food Quality Sentiment Service Sentiment Ambiance Sentiment  
0                                                              
1                                                              
2                                                              
3                                                              
4                                                              


#### Encode labels

In [ ]:
# from sklearn.model_selection import train_test_split

# # Map sentiment labels to integers
# label_map = {'positive': 2, 'neutral': 1, 'negative': 0}

# # Apply the mapping to each aspect column
# data['Food Quality Sentiment'] = data['Food Quality Sentiment'].map(label_map)
# data['Service Sentiment'] = data['Service Sentiment'].map(label_map)
# data['Ambiance Sentiment'] = data['Ambiance Sentiment'].map(label_map)

# # Drop any rows where an aspect is not labeled yet (e.g., rows with NaNs in the sentiment columns)
# data = data.dropna(subset=['Food Quality Sentiment', 'Service Sentiment', 'Ambiance Sentiment']).reset_index(drop=True)

# # Split the data into training and testing sets
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# # Display a sample from the training set for verification
# print(train_data.head())


import re

# Define keywords for each aspect
food_keywords = ['food', 'taste', 'flavor', 'meal', 'dish', 'cuisine']
service_keywords = ['service', 'staff', 'waiter', 'waitress', 'attentive', 'slow', 'friendly']
ambiance_keywords = ['ambiance', 'atmosphere', 'environment', 'decor', 'vibe', 'music']

# Function to extract aspect-specific text based on keywords
def extract_aspect_text(review, keywords):
    sentences = re.split(r'[.!?]', review)  # Split review into sentences
    aspect_text = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in keywords)]
    return ' '.join(aspect_text)

# Apply extraction for each aspect
data['Food Text'] = data['Review'].apply(lambda x: extract_aspect_text(x, food_keywords))
data['Service Text'] = data['Review'].apply(lambda x: extract_aspect_text(x, service_keywords))
data['Ambiance Text'] = data['Review'].apply(lambda x: extract_aspect_text(x, ambiance_keywords))

# Display the data to verify extraction
print(data[['Review', 'Food Text', 'Service Text', 'Ambiance Text']].head())


                                              Review  \
0  The ambience was good, food was quite good . h...   
1  Ambience is too good for a pleasant evening. S...   
2  A must try.. great food great ambience. Thnx f...   
3  Soumen das and Arun was a great guy. Only beca...   
4  Food is good.we ordered Kodi drumsticks and ba...   

                                           Food Text  \
0        The ambience was good, food was quite good    
1                                       Food is good   
2                          great food great ambience   
3   Only because of their behavior and sincerety,...   
4                                       Food is good   

                                        Service Text  \
0  \nWaiter Soumen Das was really courteous and h...   
1   Service is very prompt  Soumen Das - kudos to...   
2        Thnx for the service by Pradeep and Subroto   
3                                                      
4                                             

In [11]:
from transformers import pipeline

# Initialize a sentiment analysis pipeline using a pre-trained model
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", truncation=True)

# Modify the function to handle truncation
def analyze_sentiment(aspect_text):
    # Limit input to 512 characters to avoid exceeding model limits
    aspect_text = aspect_text[:512] if len(aspect_text) > 512 else aspect_text
    
    if aspect_text.strip():  # Ensure there is text to analyze
        result = sentiment_analyzer(aspect_text)[0]
        sentiment = result['label'].lower()
    else:
        sentiment = 'neutral'  # If no text is extracted for the aspect
    return sentiment

# Apply the sentiment analysis model to each aspect
data['Food Quality Sentiment'] = data['Food Text'].apply(analyze_sentiment)
data['Service Sentiment'] = data['Service Text'].apply(analyze_sentiment)
data['Ambiance Sentiment'] = data['Ambiance Text'].apply(analyze_sentiment)

# Display a sample to check the results
print(data[['Review', 'Food Quality Sentiment', 'Service Sentiment', 'Ambiance Sentiment']].head())


                                              Review Food Quality Sentiment  \
0  The ambience was good, food was quite good . h...               positive   
1  Ambience is too good for a pleasant evening. S...               positive   
2  A must try.. great food great ambience. Thnx f...               positive   
3  Soumen das and Arun was a great guy. Only beca...               positive   
4  Food is good.we ordered Kodi drumsticks and ba...               positive   

  Service Sentiment Ambiance Sentiment  
0          positive            neutral  
1          positive            neutral  
2          negative           positive  
3           neutral            neutral  
4           neutral            neutral  


In [12]:
# Save the dataset with aspect sentiments to a CSV file
data[['Review', 'Food Quality Sentiment', 'Service Sentiment', 'Ambiance Sentiment']].to_csv('Restaurant_Reviews_with_Aspect_Sentiment.csv', index=False)

print("Processed data saved as 'Restaurant_Reviews_with_Aspect_Sentiment.csv'")


Processed data saved as 'Restaurant_Reviews_with_Aspect_Sentiment.csv'


In [1]:
import re
import pandas as pd
from transformers import pipeline

# Define keywords for each aspect
food_keywords = ['food', 'taste', 'flavor', 'meal', 'dish', 'cuisine']
service_keywords = ['service', 'staff', 'waiter', 'waitress', 'attentive', 'slow', 'friendly']
ambiance_keywords = ['ambiance', 'atmosphere', 'environment', 'decor', 'vibe', 'music']

# Initialize the sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", truncation=True)

# Function to extract aspect-specific text based on keywords
def extract_aspect_text(review, keywords):
    sentences = re.split(r'[.!?]', review)  # Split review into sentences
    aspect_text = [sentence for sentence in sentences if any(keyword in sentence.lower() for keyword in keywords)]
    return ' '.join(aspect_text)

# Function to analyze sentiment for a given aspect text
def analyze_sentiment(aspect_text):
    aspect_text = aspect_text[:512] if len(aspect_text) > 512 else aspect_text  # Limit input to 512 characters
    
    if aspect_text.strip():  # Ensure there is text to analyze
        result = sentiment_analyzer(aspect_text)[0]
        sentiment = result['label'].lower()
    else:
        sentiment = 'neutral'  # Default to 'neutral' if no text found
    return sentiment

# Main function to process a new review
def analyze_new_review(review):
    # Extract text for each aspect
    food_text = extract_aspect_text(review, food_keywords)
    service_text = extract_aspect_text(review, service_keywords)
    ambiance_text = extract_aspect_text(review, ambiance_keywords)

    # Perform sentiment analysis on each aspect
    food_sentiment = analyze_sentiment(food_text)
    service_sentiment = analyze_sentiment(service_text)
    ambiance_sentiment = analyze_sentiment(ambiance_text)

    # Print the results
    print("Review:", review)
    print("Food Quality Sentiment:", food_sentiment)
    print("Service Sentiment:", service_sentiment)
    print("Ambiance Sentiment:", ambiance_sentiment)

    # Return the results as a dictionary (optional)
    return {
        "Review": review,
        "Food Quality Sentiment": food_sentiment,
        "Service Sentiment": service_sentiment,
        "Ambiance Sentiment": ambiance_sentiment
    }

# Example Usage
new_review = "The food was fantastic, but the service was slow, and the ambiance could have been better."
result = analyze_new_review(new_review)

# Display the result
print(result)


C:\Users\Iflal\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Review: The food was fantastic, but the service was slow, and the ambiance could have been better.
Food Quality Sentiment: negative
Service Sentiment: negative
Ambiance Sentiment: negative
{'Review': 'The food was fantastic, but the service was slow, and the ambiance could have been better.', 'Food Quality Sentiment': 'negative', 'Service Sentiment': 'negative', 'Ambiance Sentiment': 'negative'}
